In [16]:
import numpy as np
import json
from collections import Counter
from tqdm import tqdm
import nltk
import re
import string
from nltk.corpus import stopwords
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import os
import json
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU, Bidirectional, TimeDistributed, GRU, AveragePooling1D, Reshape, GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import  Bidirectional,Input,Dropout
from keras.layers import Input, Dense, Dropout, Reshape, Flatten,merge,concatenate
from keras import Model
from keras.layers import concatenate,dot
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import multi_gpu_model
from rnnlayer import Attention, SimpleAttention
from keras import backend as K
from keras.regularizers import l2
from keras.callbacks import *
# from visualizer import *
from keras.models import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical#, accuracy
from keras.layers.core import *
from keras.layers import Input, Embedding, LSTM, Dense, merge, TimeDistributed, Recurrent

import tensorflow as tf
import keras
def word_tokenize(tokens):
    return [token.replace("''", '"').replace("``", '"') for token in nltk.word_tokenize(tokens)]


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc((s))))




def get_2d_spans(text, tokenss):
    spanss = []
    cur_idx = 0
    for tokens in tokenss:
        spans = []
        for token in tokens:
            if text.find(token, cur_idx) < 0:
                print(tokens)
                print("{} {} {}".format(token, cur_idx, text))
                raise Exception()
            cur_idx = text.find(token, cur_idx)
            spans.append((cur_idx, cur_idx + len(token)))
            cur_idx += len(token)
        spanss.append(spans)
    return spanss


def get_word_span(context, wordss, start, stop):
    spanss = get_2d_spans(context, wordss)
    idxs = []
    for sent_idx, spans in enumerate(spanss):
        for word_idx, span in enumerate(spans):
            if not (stop <= span[0] or start >= span[1]):
                idxs.append((sent_idx, word_idx))
    #print(spanss)
    #print(start,stop)
    #print(context[start:stop])
    assert len(idxs) > 0, "{} {} {} {}".format(context, spanss, start, stop)
    return idxs[0], (idxs[-1][0], idxs[-1][1] + 1)
def get_word_idx(context, wordss, idx):
    spanss = get_2d_spans(context, wordss)
    return spanss[idx[0]][idx[1]][0]



def process_tokens1(tokens):
   
    
    tokens = [w for w in tokens if w not in set(string.punctuation)]
   # stop_words = set(stopwords.words('english'))
    #tokens = [w for w in tokens if not w in stop_words]
    
    return tokens



def process_tokens(temp_tokens):
    tokens = []
    
    for token in temp_tokens:
        flag = False
        l = ("-", "\u2212", "\u2014", "\u2013", "/", "~", '"', "'", "\u201C", "\u2019", "\u201D", "\u2018", "\u00B0")
        # \u2013 is en-dash. Used for number to nubmer
        # l = ("-", "\u2212", "\u2014", "\u2013")
        # l = ("\u2013",)
        tokens.extend(re.split("([{}])".format("".join(l)), token))
    
    return tokens

def generatedateset(sentence_list,question_list):
    s_list=[]
    q_list=[]
    a_list=[]
    res=[]
    
    for q in question_list:
        if(len(q)==0):
            continue
        ai=q[0][0]
      
        pi=q[0][1]
        si=q[0][2]
        
        for i in range(len(sentence_list[ai][pi])):
            if(i==si):
                res+=[[q[0][3],sentence_list[ai][pi][i],q[0][4], q[0][5]]]
            else:
                res+=[[q[0][3],sentence_list[ai][pi][i],0, q[0][5]]]
    return res

def process(data):
    q_trn=[]
    s_trn=[]
    y_trn=[]
    q_id=[]
    for i in range(len(data)):
       # tokens = [w for w in tokens if w not in set(string.punctuation) ]
        
        
        question=list(filter(None, data[i][0]))#filter empty strin
        sentence=list(filter(None, data[i][1]))#filter empty strin
        index=data[i][2]
       # list1+=[[question,sentence,index]]
        
        q_trn+=[question]
        s_trn+=[sentence]
        y_trn+=[index]
        q_id+=[data[i][3]]
    return q_trn,s_trn,y_trn, q_id

def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=51):
            continue
        try:
            embed[(line[0])] = np.array(list(map(float, line[1:])))
        except:
            pass
    
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed

def load_data(data_type):
    source_path = "{}.json".format(data_type)
    source_data = json.load(open(source_path, 'r'))
    context_list=[]
    sentence_list=[]
    question_list=[]
    answer_list=[]
    answer_index=[]
    label_index=[]
    label=[]
    idxs=[]
    word_counter = Counter()

    for ai, article in enumerate(tqdm(source_data['data'])):
        s=[]
        p = []
        for pi, para in enumerate(article['paragraphs']):
            # wordss
            context = para['context']
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')

            si = list(map(word_tokenize, sent_tokenize(context)))
           # print(si)
            si = [process_tokens(tokens) for tokens in si]  # process tokens
            for sentence in si:
                for word in sentence:
                     word_counter[word] += 1

            # given xi, add chars
            s.append(si)
            p.append(context)

            indexi = [ai, pi]
            
            for qa in para['qas']:
                    qi = word_tokenize(qa['question'])
                    if qa['is_impossible']:
                        labeli=[0]
                      #  print(labeli)
                    else:
                        labeli=[1]       
                    yi = []
                    yyi=[] 
                    answers = []
                    q_id = qa['id']
                    
                    
                    ans=[]
                    if labeli[0] == 1:
                        ans=qa['answers']
                    else:
                        ans=qa['plausible_answers']
                    for answer in ans:
                        answer_text=answer['text']
                        answers.append(answer_text)
                        answer_start = answer['answer_start']
                        answer_stop = answer_start + len(answer_text)
                       # print(context)
                       # print(si)
                        yi0, yi1 = get_word_span(context, si, answer_start, answer_stop)
                        #w0 = xi[yi0[0]][yi0[1]]            
                        #w1 = xi[yi1[0]][yi1[1]-1]
                        yi.append([ai,pi,yi0[0],qi,labeli[0],q_id]) 
                       # print(yi)
                       # print(yi)
                        yyi.append([answer_start,answer_stop])
                    for qij in qi:

                                word_counter[qij] += 1
                    question_list.append(qi)
                    answer_list.append(answers)
                    answer_index.append(yi)
                    label.append(labeli)
                    label_index.append(indexi)
                   # ids.append(qa['id'])
                    idxs.append(len(idxs))

        sentence_list.append(s)

        context_list.append(p)
        
    
    a = {'word_counter':word_counter,'sentence_list':sentence_list,'answer_index':answer_index}
    with open("{}_saved.json".format(data_type), "w") as fp:
        json.dump(a , fp) 
    print("saved in json")
    return 
def load_from_json(data_type):
    with open("{}_saved3.json".format(data_type), "r") as fp:
            a=json.load(fp) 
  
    return a['word_counter'],a['context_list'],a['sentence_list'],a['answer_index']
def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=301):
            continue
        try:
            embed[(line[0])] = np.array(list(map(float, line[1:])))
        except:
            pass
    
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed

def make_output(output):
    if output == 0:
        return torch.tensor([0])
    else:
        return torch.tensor([1])
def create_emb_layer(weights_matrix, num_embeddings,trainable=False):
    _, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    
    
    #emb_layer.load_state_dict({'weight': weights_matrix})
    
    if not trainable:
        emb_layer.weight.requires_grad = False
    else:
        emb_layer.weight.requires_grad = True
    emb_layer.weight.data.copy_(weights_matrix)
    return emb_layer, embedding_dim


def evaluate_data(preds, correct_mapping):
    
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0
    
    
    for id, pred in enumerate(correct_mapping):
        if pred == 0 and preds[id] == 0:
            true_negatives += 1
        elif pred == 1 and preds[id] == 1:
            true_positives += 1
        elif pred == 0 and preds[id] == 1:
            false_positives += 1
        elif pred == 1 and preds[id] == 0:
            false_negatives += 1
    print( true_positives,true_negatives,false_positives,false_negatives)
    precision = calculate_precision(true_positives, false_positives)
    recall = calculate_recall(true_positives, false_negatives)
    f_1 = calculate_f1(precision, recall)
    acc=(true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    return precision, recall, f_1,acc

def get_acc(preds,gold_labels):
   correct=0;
   total=len(preds)
   for i in range(total):
       if preds[i]==gold_labels[i]:
           correct+=1
   return correct/total


def calculate_precision(true_positives, false_positives):
    return true_positives / (true_positives + false_positives)


def calculate_recall(true_positives, false_negatives):
    return true_positives / (true_positives + false_negatives)


def calculate_f1(precision, recall):
    return (2 * precision * recall) / (precision + recall)


def get_we_slist(slist, word2idx):
    res_slist=[]
    for sent in slist:
        s=[]
        for word in sent:
            try:
                s.append(word2idx[word.lower()])
            except:
                s.append(word2idx['<UNK>'])
        res_slist.append(s)
                
    return res_slist
 
def create_prediction_file(preds, q_ids, fname):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    with open(fname, 'w') as outfile:
        json.dump(result, outfile)
    
def get_prediction_dict(preds, q_ids):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    return result
#create_embedding_matrix
def create_embeddingmatrix(vocab):
    
    initW = torch.nn.init.xavier_normal_(torch.randn([len(vocab), 301]))
    for i,v in enumerate(vocab):
        try:
            initW[i]=embed[v]
        except:
            continue
    embededing_matrix=initW
    return embededing_matrix
def generatedateset2(para_list,question_list):
   
    res=[]
    
    for q in question_list:
       
        ai=q[0][0]
      
        pi=q[0][1]
        si=q[0][2]
        
        res+=[[para_list[ai][pi],' '.join(q[0][3]),q[0][4],q[0][5]]]
       
    return res

def create_index_dict(embed,word_counter):
    word2idx={}
    idx2word={}
    i=1
    #index 0 is to map padded elements
    word2idx['<padded>']=0
    idx2word[0]='<padded>'
    
    for (key,value) in (embed.items()):
        word2idx[key]=i
        idx2word[i]=key
   #     vocab=np.append(vocab,[key])
        i+=1
    
    
    for key,value in (word_counter.items()):
        key=key.lower()
    
        try:
            embed[key]
           
        except: 
            try:
                word2idx[key]
            except:
                word2idx[key]=i
                idx2word[i]=key
        #        vocab=np.append(vocab,[key])
                i+=1
    
    word2idx['<concat>']=i
    idx2word[i]=['<concat>']
    i+=1
    idx2word[i]=['<unk>']
    word2idx['<unk>']=i
   # assert len(word2idx.keys())==len(vocab)
    return word2idx,idx2word,set(word2idx.keys())

def create_prediction_file(preds, q_ids, fname):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    with open(fname, 'w') as outfile:
        json.dump(result, outfile)
    
def get_prediction_dict(preds, q_ids):
    if(len(preds) != len(q_ids)):
        print("Error in pred len")
        return
    result={}
    for i in range(len(preds)):
        result[q_ids[i]] = preds[i]
        
    return result
#create_embedding_matrix
# def create_embeddingmatrix(vocab):
    
#     initW = torch.nn.init.xavier_normal_(torch.randn([len(vocab), 301]))
#     for i,v in enumerate(vocab):
#         try:
#             initW[i]=embed[v]
#         except:
#             continue
#     embededing_matrix=initW
#     return embededing_matrix



def create_embeddingmatrix(word2idx):
   
    vocab_list=list(word2idx.keys())
    initW = np.random.randn(len(word2idx)+1 ,300)
    
    for i,v in enumerate(vocab_list):
        try:
            initW[i]=embed[v]
        except:
            if v=='<padded>' or v=='<unk>':
                initW[i]=np.zeros(300)

            continue
    embededing_matrix=initW
    return embededing_matrix

def find_index(dict1,word):
  #  print(word)
    word=word.lower()
    try:
        a= dict1[word]
    except:
        filterd_word=re.sub(r"[^AZa-z1-9]",r"",word)
        try:
            a= dict1[filterd_word]
        except:
            a= dict1['<unk>']
    return a

    
    

def padded_sentences(slist,length):
    
    n=len(slist)
    result=np.zeros((n,length))
    for i in range(n):
        for jj,j in enumerate(slist[i]):
            if(jj>=length):
                break
            result[i][jj]=find_index(word2idx,j)
           
    return result

def combine_pq(p,q):
   assert len(p)==len(q)
   return [p[i]+['<concat>']+q[i] for i in range(len(p))]
def generate_data(para_list,qs):
    #qmax:800
    #smax:60
    n=len(answer_index)
#     max_p=-1
#     max_q=-1
    para_=[]
    qst_=[]
    y=[]
    yid=[]
    for q in (qs):
        ai=q[0][0]
        
        pi=q[0][1]
        si=q[0][2]
        para=para_list[ai][pi]
        qst=q[0][3]
        
     #   print(len(qst))
      #  para_tokens=list(filter(None,word_tokenize(para)))
      #  qst_tokens=list(filter(None,qst))
       
#         if(len(para)>max_p):
#             max_p=len(para)
#         if(len(qst)>max_q):
#             max_q=len(qst)
        para_.append(para)
        qst_.append(qst)
        y.append(q[0][4])
        yid.append(q[0][5])
  
    return para_,qst_,np.array(y),yid


     


In [3]:
embed_file="D:/glove.6B.300d.txt"
embed=read_embedding(embed_file)
print("finish loading embedding file")


[D:/glove.6B.300d.txt]
	Embedding size: 400000
finish loading embedding file


In [4]:

data_type='training'
word_counter,para_list,sentence_list,answer_index=load_from_json(data_type)
data_type='development'
d_word_counter,d_para_list,d_sentence_list,d_answer_index=load_from_json(data_type)
nltk.data.path.append("/afs/cs.stanford.edu/u/tianzhao/tz/nltk_data")
sent_tokenize = nltk.sent_tokenize

#X=generatedateset2(para_list,answer_index)


In [5]:
ptrn,qtrn,ytrn,trn_id=generate_data(para_list,answer_index)
word2idx,idx2word,vocab=create_index_dict(embed,word_counter)
ptrn_pad=padded_sentences(ptrn,600)
qtrn_pad=padded_sentences(qtrn,30)

combined=combine_pq(ptrn,qtrn)
combined_pad=padded_sentences(combined,700)

#embed['<padded>']=np.zeros(300)
embedding_matrix=create_embeddingmatrix(word2idx)

    #preocess devset
pdev,qdev,ydev,dev_id=generate_data(d_para_list,d_answer_index)
#word2idx,idx2word,vocab=create_index_dict(embed,word_counter)
pdev_pad=padded_sentences(pdev,600)
qdev_pad=padded_sentences(qdev,30)

combined_dev=combine_pq(pdev,qdev)
combined_pad_dev=padded_sentences(combined_dev,700)

In [6]:
question_maxlen=30
context_maxlen=600
EMBEDDING_DIM=300
vocab_size=len(vocab)+1

In [18]:


config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)



question_input = Input(shape=(question_maxlen,), dtype='int32', name='question_input')
#question_input = Input(batch_shape=(None,), dtype='float32', name='question_input')
context_input = Input(shape=(context_maxlen,), dtype='int32', name='context_input')
#context_input = Input(batch_shape=(None,), dtype='float32', name='context_input')

questionEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         mask_zero=False, weights=[embedding_matrix], 
                         input_length=question_maxlen, trainable=False)(question_input)
contextEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,
                         mask_zero=False, weights=[embedding_matrix], 
                         input_length=context_maxlen, trainable=False)(context_input)

Qst = Bidirectional(GRU(32, return_sequences=True))(questionEmbd)
C = Bidirectional(GRU(32, return_sequences=True))(contextEmbd)
#Q1 = Bidirectional(GRU(, return_sequences=True))(Q)
#D1 = Bidirectional(GRU(96, return_sequences=True))(D)
Qst2 = Bidirectional(GRU(64, return_sequences=False))(Qst)
C2 = Bidirectional(GRU(64, return_sequences=False))(C)
C2 = SimpleAttention(128, Q2, 128, return_sequences=False)(C)
L=concatenate([C2,Qst2],axis=-1)

y= Dense(1, activation='sigmoid')(L)

model = Model(inputs=[context_input, question_input], outputs=[y])
rms = keras.optimizers.RMSprop(lr=0.0005)
checkpoint1 = ModelCheckpoint("checkpoint/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=1, save_best_only=False, mode='auto',save_weights_only=False,period=1)
checkpoint3=ReduceLROnPlateau(patience=3, monitor='val_loss',min_lr=1e-7,factor=0.7,verbose=1)
model.compile(optimizer=rms, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 600)          0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 600, 300)     125828400   context_input[0][0]              
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 30, 300)      125828400   question_input[0][0]             
__________________________________________________________________________________________________
bidirectio

In [ ]:
model.fit([ptrn_pad, qtrn_pad], [ytrn], validation_data=[[pdev_pad,qdev_pad],ydev],epochs=30, batch_size=128, shuffle=True,callbacks=[checkpoint1,checkpoint2,checkpoint3])

Train on 69596 samples, validate on 17400 samples
Epoch 1/30
69596/69596 [==============================] - 1424s 20ms/step - loss: 0.6698 - acc: 0.5826 - val_loss: 0.6530 - val_acc: 0.6104

Epoch 00001: saving model to checkpoint/weights.01-0.65.hdf5
Epoch 2/30
69596/69596 [==============================] - 1456s 21ms/step - loss: 0.6463 - acc: 0.6161 - val_loss: 0.6415 - val_acc: 0.6224

Epoch 00002: saving model to checkpoint/weights.02-0.64.hdf5
Epoch 3/30
69596/69596 [==============================] - 1522s 22ms/step - loss: 0.6270 - acc: 0.6350 - val_loss: 0.6327 - val_acc: 0.6264

Epoch 00003: saving model to checkpoint/weights.03-0.63.hdf5
Epoch 4/30
69596/69596 [==============================] - 1489s 21ms/step - loss: 0.6056 - acc: 0.6577 - val_loss: 0.6195 - val_acc: 0.6393

Epoch 00004: saving model to checkpoint/weights.04-0.62.hdf5
Epoch 5/30
69596/69596 [==============================] - 1437s 21ms/step - loss: 0.5861 - acc: 0.6730 - val_loss: 0.6077 - val_acc: 0.6527

E

In [24]:
data_type="test"
t_word_counter,t_para_list,t_sentence_list,t_answer_index=load_from_json(data_type)

In [36]:
ptst,qtst,tst_id=generate_data(t_para_list,t_answer_index)
ptst_pad=padded_sentences(ptst,600)
qtst_pad=padded_sentences(qtst,30)

In [175]:
weights_path="checkpoint1/weights.01-0.65.hdf5"
model.load_weights(weights_path)

In [100]:
pred1=model.predict([ptst_pad,qtst_pad])
pred=inference(pred1,0.5) 
evaluate_data(pred,ydev)

4809 7259 1441 3891


(0.76944, 0.5527586206896552, 0.6433444816053512, 0.6935632183908046)

In [144]:

result=list(zip(tst_id,pred))

import pandas as pd
labels = ['Id','Category']
df = pd.DataFrame.from_records(result, columns=labels)
df.to_csv("lr_sample8.csv",index=False,header=True,sep=",")
print('done')

done
